#  Poppy-Rattle


The Poppy Rattle is a project that is focused on modeling the effects of toy interaction on infant development.
For example, if given a rattle to shake, how does an infant develop motor skill such as rhythm? Are there any effects on an infants reactionss to sounds in general, and perhaps is there any effect on early language development? All of these questions are relevent to this project.

This version uses the working Poppy-Rattle library. Using the poppy format, it can be be switched between physical and simulated mode by passing an argument in the initialization.

In [10]:
%pylab inline

from __future__ import division # the python 3 division - outputs double instead of am int
from __future__ import print_function # the python 3 print function - print()
from tqdm import tqdm # library add a progrees bar for an iterating for loops

from explauto.environment import environments # library environment for modeling implementation
from librosa.feature import melspectrogram

import time,datetime # librraies to keep track of times
import threading  # library for multithreading workloads

import os    #
import wave  # library for recording sound and exporting as a wave file
import alsaaudio, audioop, sys, librosa  # library for recording sound

import numpy as np # python's standard linear algebra library: needed for static arrays
import matplotlib.pyplot as plt # library for graphing data
import torch as th    # a python version of Lua's Torch ML library.

from poppy.creatures import PoppyRattle
poppy = PoppyRattle()    # creates poppy object. For physical robot 
# poppy = PoppyRattle(simulator='vrep')    # creates poppy object. For simulated robot. vrep scene needs work

# for loop primes the motors to be programmed
for m in poppy.motors:
    m.compliant = False
    #m.goto_behavior = 'minjerk' # prevents jerky movements

# Python (array) lists to store feedback data

# for physical robot
pos = []
# for simulator
pos_Head = []    # Stores the position of the hand (wrist) relative to the head
pos_Stand = []    # Stores the position of the hand (wrist) relative to the center point between the feet
TIME = []    # Stores timestamps of when the arm is in a certain position
sys_load = []    # Stores the torque the motors are going through at a point in time.
                 # Good for being warry of the system load of a task and possibly calculating fatigue
speed = []    # Stores the angle speed the motors are travelling at
temp = []   # Stores the motors' temperature
volt = []   # Stores the motor's voltage. 
            # In conjunction with Temperaure, this can be used to measure energy of a task

# specific arrays for the rattle shaking movement
data_table = []  # stores the sound features of a recording
spike = [[],[]]  # stores instances of abnormal spikes in sound features and a given timestamp


# activate microphone - A Logitech C920 webcam
inp = alsaaudio.PCM(alsaaudio.PCM_CAPTURE,alsaaudio.PCM_NONBLOCK,device='sysdefault:CARD=C920')
inp.setchannels(2)
inp.setrate(88200)
inp.setformat(alsaaudio.PCM_FORMAT_GSM)
inp.setperiodsize(160)

Populating the interactive namespace from numpy and matplotlib


2822L

# Motors

poppy.r_shoulder_x:
    * id: 52
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -100.0 to 0.0 degrees 

poppy.r_shoulder_y
    * id: 51
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -50.0 to 170.0 degrees 
    
poppy.r_arm_z
    * id: 53
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -90.0 to 90.0 degrees 

poppy.r_elbow_y
    * id: 54
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: 0.0 to 130.0 degrees 

## A number of function commands for the robot

These will be added to primitves down the line.

In [19]:
def rest_position():
    """
    This command function returns the arm to a "resting" position, although it is more on the 
    lines of the attention position.
    
    For the physical arm, it is important to calibrate the motors so that the resting position 
    is set for all the motors is at angle 0. The simulator defaults to this, so this will 
    eliminate the need to constantly translate angles when switching between the two.
    """
    poppy.r_shoulder_x.goal_position = 0   
    poppy.r_shoulder_y.goal_position = 0
    poppy.r_arm_z.goal_position = 0
    poppy.r_elbow_y.goal_position = 0
    

def app():
    """
    This function merely appends system sensory information into the appropriate python list.
    """
    
    #
    if poppy.simulated == True:
        # appends wrist position relative to head position
        pos_Head.append(poppy.get_object_position('r_forearm_visual','head_visual'))
        # appends wrist position relative to the center point between the feet
        pos_Stand.append(poppy.get_object_position('r_forearm_visual')) 
    else:
        # stores the angle position of all the motors
        pos.append(poppy.r_shoulder_x.present_position)
        pos.append(poppy.r_shoulder_y.present_position)
        pos.append(poppy.r_arm_z.present_position)
        pos.append(poppy.r_elbow_y.present_position)
    
    # appends the stress each motor is going through
    sys_load.append(poppy.r_shoulder_x.present_load)
    sys_load.append(poppy.r_shoulder_y.present_load)
    sys_load.append(poppy.r_arm_z.present_load)
    sys_load.append(poppy.r_elbow_y.present_load)
   
    # appends the angle speed each motor is currently traveling at 
    speed.append(poppy.r_shoulder_x.present_speed)
    speed.append(poppy.r_shoulder_y.present_speed)
    speed.append(poppy.r_arm_z.present_speed)
    speed.append(poppy.r_elbow_y.present_speed)
    
    # appends system temperature for each motor
    temp.append(poppy.r_shoulder_x.present_temperature)
    temp.append(poppy.r_shoulder_y.present_temperature)
    temp.append(poppy.r_arm_z.present_temperature)
    temp.append(poppy.r_elbow_y.present_temperature)
    
    # appends the voltage ruuning through each motor
    volt.append(poppy.r_shoulder_x.present_voltage)
    volt.append(poppy.r_shoulder_y.present_voltage)
    volt.append(poppy.r_arm_z.present_voltage)
    volt.append(poppy.r_elbow_y.present_voltage)


def hand_wave(sec=10, rest=0.5):
    """
    This function is a primitive. When called, the robotic arm will move between two set point in space, defined by the angles the motors are travelling to.
    
        :param int sec: time in seconds for how long the arm will wave
            ::default is set to 10 seconds
        :param float rest: time in seconds for the time the arm has to finish its motion
            ::default is set to 1/2 a second 
    """
    
    t0 = time.time()   # Gets the timestamp at the start of the waving motion
    
    
    # This will loop through the two arm positions until the set time duration has elapsed. 
    while True:
        t1 = time.time() # Gets the timestamp at the end of each loop
        
        if t1-t0 >= sec: # This will end the loop and wave motion when the allotted time has expired
            break
            
        
        # run for sec
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 30
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -135
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = -120;
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 0
        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
    
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -175
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = -120
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = -65
        
        app()
        TIME.append(t1-t0)
        
        time.sleep(rest)
        app()
        TIME.append(t1-t0)
        
        
  
        
# function makes the arm shake a rattle
def rattle_shake(sec=20, rest=.4):
    
    # define the first point in space
    shoulder_y0 = -45
    shoulder_x0 = 0
    arm_0 = 0
    elbow_0 = -65
    
    # define the second point in space
    shoulder_y1 = -30
    shoulder_x1 = 0
    arm_1 = 0
    elbow_1 = -35
    
    # speed adjustments to be more infant like
    poppy.r_shoulder_y.moving_speed = abs(shoulder_y1-shoulder_y0) / rest
    poppy.r_shoulder_x.moving_speed = abs(shoulder_x1-shoulder_x0) / rest
    poppy.r_arm_z.moving_speed = abs(arm_1-arm_0) / rest
    poppy.r_elbow_y.moving_speed = abs(elbow_1-elbow_0) / rest

    t0 = time.time()
    while True:
              
        t1 = time.time() # this time gets updated at each loop
        if t1-t0 >= sec:  # difference between t1 and t0 indicates elapsed time
            break  # when time elapsed exceeds alotted runtime loop is exited
            
        

        # arm moves to first position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y0 #-45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x0 #5
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_0 #20
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_0 #-129
        

        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        # arm moves to second position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y1 #-25
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x1  #0
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_1; #0
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_1 #-10
        


       
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        
        app()
        TIME.append(t1-t0)
    

    
    # resets motor speed
    for x in poppy.motors:
            x.moving_speed = 90

    
def rattle_shake(sec=5, rest=0.4):
    """
    This function is a primitive. When called, the robotic arm will move between two set point
    in space, defined by the angles the motors are travelling to. This will make the arm 
    perform a vertical movement in order to imitate an infant shaking a rattle.

        :param int sec: time in seconds for how long the arm will wave
            ::default is set to 20 seconds
        :param float rest: time in seconds for the time the arm has to finish its motion
            ::default is set to 0.4 seconds
    """
    
    # define the first point in space
    shoulder_y0 = -45
    shoulder_x0 = 0
    arm_0 = 0
    elbow_0 = -65
    
    # define the second point in space
    shoulder_y1 = -30
    shoulder_x1 = 0
    arm_1 = 0
    elbow_1 = -35
    
    # speed adjustments to be more infant like
    poppy.r_shoulder_y.moving_speed = 100
    poppy.r_shoulder_x.moving_speed = 100
    poppy.r_arm_z.moving_speed = 100
    poppy.r_elbow_y.moving_speed = 100

    
    
    t0 = time.time() # Gets the timestamp at the start of the waving motion
    
    # This will loop through the two arm positions until the set time duration has elapsed. 
    while True:
        t1 = time.time() # Gets the timestamp at the start of each loop
        
        # This will end the loop and wave motion when the allotted time has expired
        if t1-t0 >= sec: 
            break
            
        # arm moves to first position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y0 #-45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x0 #5
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_0 #20
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_0 #-129
        

        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        # arm moves to second position
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = shoulder_y1 #-25
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = shoulder_x1  #0
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = arm_1; #0
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = elbow_1 #-10
        


       
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
        
        
        app()
        TIME.append(t1-t0)
    
         


    
    # resets motor speed
    for x in poppy.motors:
            x.moving_speed = 90

In [36]:
poppy.r_shoulder_y.goal_position = -30


In [35]:
poppy.config

{u'controllers': {u'upper_body_controller': {u'attached_motors': [u'r_arm'],
   u'port': u'auto',
   u'sync_read': False}},
 u'motorgroups': {u'r_arm': [u'r_shoulder_y',
   u'r_shoulder_x',
   u'r_arm_z',
   u'r_elbow_y']},
 u'motors': {u'r_arm_z': {u'angle_limit': [-90, 90],
   u'id': 53,
   u'offset': 0.0,
   u'orientation': u'direct',
   u'type': u'MX-28'},
  u'r_elbow_y': {u'angle_limit': [0, 130],
   u'id': 54,
   u'offset': 0.0,
   u'orientation': u'indirect',
   u'type': u'MX-28'},
  u'r_shoulder_x': {u'angle_limit': [-100, 0],
   u'id': 52,
   u'offset': 0.0,
   u'orientation': u'indirect',
   u'type': u'MX-28'},
  u'r_shoulder_y': {u'angle_limit': [-50, 170],
   u'id': 51,
   u'offset': 0,
   u'orientation': u'indirect',
   u'type': u'MX-28'}}}

In [12]:
# audio recording setup
# creates a new wavefile for each run
def outFile():
    return "../out_Data/rattle_{}.wav".format(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S'))

In [31]:
# function causes the rattle to shake and record audio
def alsa_rattle(function,outFile):
    # set up the wave file
    w = wave.open(outFile,'w')
    # Open the device in nonblocking capture mode. The last argument could
    # just as well have been zero for blocking mode. Then we could have
    # left out the sleep call in the bottomvim of the loop

    w.setnchannels(2)
    w.setsampwidth(2)
    w.setframerate(44100)

    total = 0
    
    # allows function to run in parallel
    t = threading.Thread(target=function)
    t.start()
    timeStart = time.time()
    while t.is_alive():
        timeStop = time.time()
        l,data = inp.read()
        if l:
            # Return the maximum of the absolute value of all samples in a fragment.
            frame = audioop.max(data,2)
            data_table.append(frame)
            total += frame
            ave = total / len(data_table)
            if(len(data_table) > 25) :
                if (frame > ave * 1.34):
                    spike[0].append(timeStop - timeStart)
                    spike[1].append(frame)
            w.writeframes(data)
            # print audioop.max(data, 2)
        time.sleep(.001)
    rest_position()

## Resets the arrays and returns the arm to the resting position

Useful for getting data on different motions independently.

In [21]:
rest_position()

pos_Head = []
pos_Stand = []
pos = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []
data_table = []  
spike = [[],[]] 

### Calling movement functions

In [33]:
# Run to to make the arm wave
hand_wave()
time.sleep(0.25) # Gives the arm a little time to complete a motion before commencing the next action
rest_position()  # Optional: returns the arm to the resting position

In [32]:
# Run to make the arm perform a rattle shake movement
out = outFile()
alsa_rattle(rattle_shake, out)
time.sleep(0.25) # Gives the arm a little time to complete a motion before commencing the next action
rest_position()  # Optional: returns the arm to the resting position

In [ ]:
melspectrogram

In [ ]:
import torch as th


In [17]:
poppy.r_shoulder_y.goal_position = -160
poppy.r_shoulder_x.goal_position = -10
poppy.r_arm_z.goal_position = -10;


In [18]:
poppy.config

{u'controllers': {u'upper_body_controller': {u'attached_motors': [u'r_arm'],
   u'port': u'auto',
   u'sync_read': False}},
 u'motorgroups': {u'r_arm': [u'r_shoulder_y',
   u'r_shoulder_x',
   u'r_arm_z',
   u'r_elbow_y']},
 u'motors': {u'r_arm_z': {u'angle_limit': [-90, 90],
   u'id': 53,
   u'offset': 0.0,
   u'orientation': u'direct',
   u'type': u'MX-28'},
  u'r_elbow_y': {u'angle_limit': [7, 147],
   u'id': 54,
   u'offset': 0.0,
   u'orientation': u'indirect',
   u'type': u'MX-28'},
  u'r_shoulder_x': {u'angle_limit': [-110, 105],
   u'id': 52,
   u'offset': 0.0,
   u'orientation': u'indirect',
   u'type': u'MX-28'},
  u'r_shoulder_y': {u'angle_limit': [-155, 120],
   u'id': 51,
   u'offset': 0,
   u'orientation': u'indirect',
   u'type': u'MX-28'}}}

## Run when experiments are done

This cell resets the robot and releases the robot.
After running this set of commands, rerunning the setup cell is necessary

### **NOTE**
If this fails or some error occurs, it may be necessary to run a hard reset:
    
    For physical robot:
        1. Unplug the robot
    
    For simulator:
        1.Close the simulator
    
    2. Save and shutdown the ipython notebook
    3. It may be a good idea to also close the jupyter-notebook server and restart from there.

In [9]:
# RUN at the end of working

pos = []
pos_Head = []
pos_Stand = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []
data_table = []
spike = [[],[]]
for m in poppy.motors:
    m.compliant = True
time.sleep(1)
poppy.stop_sync()
poppy.close()

In [31]:
poppy.r_elbow_y.goal_position = -50 #0

In [ ]:
# rms feature extraction - feed into explauto
# proportion/total power over 400hz
# mel spectrogram
# amek explauto learn the motor sounds